In [1]:
import librosa.display
import librosa.util
from IPython.display import Audio
import os
import scipy
from scipy.io import wavfile
import numpy as np
import utils.synthesis_util as su
import utils.model_util as mu
from utils.metrics import Metrics
from pesq_lib.pypesq import pypesq

## Examine dataset

In [2]:
audio_files = su.load_files()
id_audio=2386

### Clean audio

In [3]:
y_clean,sr = su.get_audio(audio_id=id_audio,audio_files=audio_files)
Audio(y_clean,rate=sr)

## Noisy audio

In [4]:
y_noise, sr = su.get_noisy_audio(audio_id=id_audio,audio_files=audio_files, noise_factor=0.05)
Audio(y_noise,rate=sr)

In [5]:
win = scipy.signal.hamming(256,"periodic")
magnitude,phase = su.audio_to_sttft(y_noise,win)

In [6]:
predictors = su.get_predictors(magnitude)

In [7]:
predictors = np.array(predictors)
predictors.shape

(306, 129, 8)

## Load module

In [8]:
model = mu.load_model(n_features=129,n_segments=8,model_to_test="saved/LSTM/0610_114004", type='lstm')

## Denoise

In [9]:
length = len(y_noise)

In [10]:
sample = su.obtain_sample(predictors)
sample = sample.view(1, sample.shape[0], sample.shape[1])

In [11]:
# print(model)
audio_rec = su.denoise_audio(model,sample,phase,win,length, type ='rnn')
print(audio_rec.shape)
Audio(audio_rec,rate=sr)

RuntimeError: Expected object of backend CUDA but got backend CPU for argument #3 'index'

### Visualize spectrograms

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import numpy as np

In [ ]:
plt.subplot(1,3,1)
S = librosa.feature.melspectrogram(y=y_clean,
                                   sr=sr,
                                   n_fft=1024,
                                   hop_length=256,
                                   n_mels=80,
                                   fmin=125,
                                   fmax=7600)
librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
plt.title("Clean")

plt.subplot(1,3,2)
S = librosa.feature.melspectrogram(y=y_noise,
                                   sr=sr,
                                   n_fft=1024,
                                   hop_length=256,
                                   n_mels=80,
                                   fmin=125,
                                   fmax=7600)
librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
plt.title("Noisy")

plt.subplot(1,3,3)
S = librosa.feature.melspectrogram(y=audio_rec,
                                   sr=sr,
                                   n_fft=1024,
                                   hop_length=256,
                                   n_mels=80,
                                   fmin=125,
                                   fmax=7600)
librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
plt.title("Denoised")
plt.show()


# Calculate metrics

In [ ]:
magnitude_clean,phase_clean = su.audio_to_sttft(y_clean,win)
magnitude_noise,phase_noise = su.audio_to_sttft(y_noise,win)
magnitude_denoised,phase_denoised = su.audio_to_sttft(audio_rec,win)
print(f"SNR_db for clean: {10*np.log10(Metrics.DER_SNR(Metrics.spectral_flux(magnitude_clean, sr)))}")
print(f"SNR_db for noise: {10*np.log10(Metrics.DER_SNR(Metrics.spectral_flux(magnitude_noise, sr)))}")
print(f"SNR_db for denoised: {10*np.log10(Metrics.DER_SNR(Metrics.spectral_flux(magnitude_denoised, sr)))}")


In [ ]:
print(sr)

In [ ]:
print(f"PESQ value with narrowband (nb) considering original signal and original+noise: {pypesq(sr, y_clean, y_noise, 'nb')}")
print(f"PESQ value with narrowband (nb) considering original signal and reconstructed signal: {pypesq(sr, y_clean, audio_rec, 'nb')}")

### Aggregated metrics

In [ ]:
trainset_indices = su.get_trainset_indices()
print(trainset_indices)
testset_inidices = su.get_testset_inidces()
print(testset_inidices)

In [ ]:
Metrics.aggregate_metric_check(audio_files, model, indices=trainset_indices)

In [ ]:
Metrics.aggregate_metric_check(audio_files, model, indices=testset_inidices)